In [9]:
config = get_config()
data_dir = Path(config['data_location'])
domain_file = 'blocks-domain-updated.pddl'
domain_file = data_dir / 'domain' / domain_file
problem_directory = 'multitower'
problem_number = 1
problem_file = data_dir / problem_directory / f'problem{problem_number}.pddl'
domain, problem = pddl_functions.parse(domain_file, problem_file)

pddl_state = pddl_functions.PDDLState.from_problem(problem)
action = pddl_functions.Action.from_pddl(domain.actions[0])

state = [('b1', 'red'), ('b2', 'blue'), ('b3', 'blue'), ('b4', 'purple'), ('b5', 'blue')]
action.apply_action(pddl_state, ['b1', 't0', 'tower0'])
action.apply_action(pddl_state, ['b2', 'b1', 'tower0'])
action.apply_action(pddl_state, ['b3', 't1', 'tower1'])
action.apply_action(pddl_state, ['b4', 'b3', 'tower1'])
action.apply_action(pddl_state, ['b7', 'b4', 'tower1'])
action.apply_action(pddl_state, ['b5', 'b7', 'tower1'])


colour_counts = defaultdict(int)
for o, c in state:
    print(o, c)
    pddl_state.apply_effect(Predicate(c, [o]))
    for t in pddl_state.towers:
        tower = t.replace('t', 'tower')
        if pddl_state.predicate_holds("in-tower", [o, tower]):
            colour_counts[(c, tower)] += 1



new_fexpressions = []
for cc in pddl_state.fexpressions:
    print(cc, colour_counts[(cc.colour, cc.tower)])
    cc = ColourCount(cc.colour, cc.tower, colour_counts[(cc.colour, cc.tower)])
    new_fexpressions.append(cc)
    print(cc)
pddl_state.fexpressions = new_fexpressions


for formula in pddl_state.to_formula():
    print(formula.asPDDL())
assert(pddl_state.predicate_holds("in-tower", ['b2', 'tower0']))

assert(pddl_state.get_colour_count('red', 'tower0') == 1)
assert(pddl_state.get_colour_count('red', 'tower1') == 0)
assert(colour_counts[('blue', 'tower1')] == 2)
assert(colour_counts[('blue', 'tower0')] == 1)
assert (pddl_state.get_colour_count('blue', 'tower0') == 1)

assert (pddl_state.get_colour_count('purple', 'tower1') == 1)

assert(pddl_state.get_colour_count('blue', 'tower1') == 2)




Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain-updated.pddl
Parsing problem /home/mappelgren/Desktop/correcting-agent/data/multitower/problem1.pddl
b1 red
b2 blue
b3 blue
b4 purple
b5 blue
(= (blue-count tower1) 1) 2
(= (blue-count tower1) 2)
(= (blue-count tower0) 0) 1
(= (blue-count tower0) 1)
(= (red-count tower1) 0) 0
(= (red-count tower1) 0)
(= (red-count tower0) 1) 1
(= (red-count tower0) 1)
(= (green-count tower1) 0) 0
(= (green-count tower1) 0)
(= (green-count tower0) 0) 0
(= (green-count tower0) 0)
(= (yellow-count tower1) 2) 0
(= (yellow-count tower1) 0)
(= (yellow-count tower0) 0) 0
(= (yellow-count tower0) 0)
(= (purple-count tower1) 0) 1
(= (purple-count tower1) 1)
(= (purple-count tower0) 0) 0
(= (purple-count tower0) 0)
(= (orange-count tower1) 0) 0
(= (orange-count tower1) 0)
(= (orange-count tower0) 1) 0
(= (orange-count tower0) 0)
(= (pink-count tower1) 1) 0
(= (pink-count tower1) 0)
(= (pink-count tower0) 0) 0
(= (pink-count tower0

In [1]:
from collections import defaultdict
from pathlib import Path

import pytest
from pythonpddl import pddl

from correctingagent.pddl import pddl_functions
from correctingagent.pddl.pddl_functions import *
from correctingagent.util.util import get_config

